In [7]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt

In [8]:
# import the data
data = pd.read_csv("clean_data.csv")
print(data.head())

   Unnamed: 0                                               body  \
0           0  يبغى التنبيه على ان السعوديه  تستخدم صواريخ جو...   
1           1  امريكا قتلت بالامس معوق رفض رفع يديه فماذا تري...   
2           2  هذا الشخص هو من كان مؤيد لاحتلال العراق وضرب ا...   
3           3  الى جمال ريان مذيع الجزيره  نحن من رعاك فى الم...   
4           4  خيبه  الامل ليست تشاؤما ولا تقولا    ٠   عزم ل...   

   languagecomment  
0               -1  
1               -1  
2               -1  
3               -1  
4                0  


In [16]:
words = data['body'].str.split(' ')

In [17]:
data.replace('\n', '<br />')

,Unnamed: 0,body,languagecomment
0,0,يبغى التنبيه على ان السعوديه تستخدم صواريخ جو...,-1
1,1,امريكا قتلت بالامس معوق رفض رفع يديه فماذا تري...,-1
2,2,هذا الشخص هو من كان مؤيد لاحتلال العراق وضرب ا...,-1
3,3,الى جمال ريان مذيع الجزيره نحن من رعاك فى الم...,-1
4,4,خيبه الامل ليست تشاؤما ولا تقولا ٠ عزم ل...,0
...,...,...,...
31687,31687,انه لا علاقه بتاتا بين البطوله و القتل الساد...,-1
31688,31688,انتقام . . ام حسد ؟,0
31689,31689,الظلم ظلمات فى الدنيا والاخره ليس انتقادا للح...,0
31690,31690,بودموس و سیریزا املان للعالم للتخلص من الراسما...,-1


In [18]:
label_data = data[['body', 'languagecomment']]
label_data[:5]

,body,languagecomment
0,يبغى التنبيه على ان السعوديه تستخدم صواريخ جو...,-1
1,امريكا قتلت بالامس معوق رفض رفع يديه فماذا تري...,-1
2,هذا الشخص هو من كان مؤيد لاحتلال العراق وضرب ا...,-1
3,الى جمال ريان مذيع الجزيره نحن من رعاك فى الم...,-1
4,خيبه الامل ليست تشاؤما ولا تقولا ٠ عزم ل...,0


In [19]:
sample = label_data[:500]

In [20]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(label_data, test_size=0.2) #sample

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from time import time

max_features = 5000

x = train['body']
y = train['languagecomment']


tfidf = TfidfVectorizer(max_features=max_features)

pipeline = Pipeline([
    ('tfidf', tfidf),
    ('NB', MultinomialNB (alpha=1)), 
])

parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2')   
}

grid_search = GridSearchCV(pipeline, parameters, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time()
grid_search.fit(x, y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
#refitting on entire training data using best settings
grid_search.refit

x_test = test['body']
y_test = test['languagecomment']

predicted = grid_search.predict(x_test)
print("accuracy=", np.mean(predicted == y_test))
print("recall=", recall_score(y_test, predicted, average='weighted') )
print("precision=", precision_score(y_test, predicted, average='weighted'))
print("weighted f-score", f1_score(y_test, predicted, average='weighted'))
print("Confusion matrix\n", confusion_matrix(y_test, predicted))

Performing grid search...
pipeline: ['tfidf', 'NB']
parameters:
{'tfidf__use_idf': (True, False), 'tfidf__norm': ('l1', 'l2')}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.6s finished


done in 6.215s

Best score: 0.814
Best parameters set:
	tfidf__norm: 'l2'
	tfidf__use_idf: True
accuracy= 0.813693011516012
recall= 0.813693011516012
precision= 0.8106403913812922
weighted f-score 0.7436105478570842
Confusion matrix
 [[  13   92    0]
 [   0 5078   19]
 [   0 1070   67]]
